In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install unrar
!unrar  x "/content/drive/MyDrive/AIMedic_Moazed/Resources/EchoNet-Dynamic_Cases.rar" -d ./

In [ ]:
!git clone --branch ejection_fraction https://github.com/iamsoroush/echotrain.git
!pip install -r echotrain/requirements_colab.txt

In [1]:
from echotrain.utils.handling_yaml import load_config_file
from echotrain.model.ejection_fraction.ejection_fraction_estimation import EFEstimation
from echotrain.evaluation.evaluator_ef import EFEvaluation
config_path = "/content/echotrain/config/config_example.yaml"
config = load_config_file(config_path)
ef_es = EFEstimation()
ef_ev = EFEvaluation(config)

In [ ]:
model = ef_es.load_model('/content/gbr_test.sav')

In [2]:
x_train , y_train = ef_ev.data_for_ftov('train')
x_val , y_val = ef_ev.data_for_ftov('val')

In [3]:
x_train.shape

(14920, 112, 112)

In [4]:
x_val.shape

(2576, 112, 112)

In [5]:
import numpy as np
x_train_rp = []
for frame in x_train:
  rp = []
  rp.append(ef_es.frame_to_rp(frame))
  x_train_rp.append(rp)
x_train = np.array(x_train_rp).reshape(-1, 6)
x_val_rp = []
for frame in x_val:
  rp = []
  rp.append(ef_es.frame_to_rp(frame))
  x_val_rp.append(rp)
x_val = np.array(x_val_rp).reshape(-1, 6)

In [7]:
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor

mlp = MLPRegressor()
lr=LinearRegression()
dtr = DecisionTreeRegressor()
gbr = GradientBoostingRegressor(n_estimators=30)
knn = KNeighborsRegressor()
randomforest=RandomForestRegressor(n_estimators = 150)
svr=SVR(kernel='rbf')
gbr.fit(x_train,y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=30,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [8]:
gbr.score(x_val,y_val)

0.5652189661181253

In [9]:
ef_ev.evaluation_of_ef_model(gbr)

/content/echotrain/evaluation/evaluator_ef.py:105: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  es_x_path = DF[DF['case_id'] == case][DF['stage'] == 'ES']['image_path'].astype('string')
/content/echotrain/evaluation/evaluator_ef.py:106: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ed_x_path = DF[DF['case_id'] == case][DF['stage'] == 'ED']['image_path'].astype('string')
/content/echotrain/evaluation/evaluator_ef.py:107: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  es_y_path = DF[DF['case_id'] == case][DF['stage'] == 'ES']['label_path'].astype('string')
/content/echotrain/evaluation/evaluator_ef.py:108: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ed_y_path = DF[DF['case_id'] == case][DF['stage'] == 'ED']['label_path'].astype('string')
/content/echotrain/evaluation/evaluator_ef.py:109: UserWarning: Boolean Series key will be reindexed to match DataFrame inde

{'mean_absoulute_error_validation': 12.83871487919965,
 'mean_squared_error_validation': 269.87871049228244}

In [ ]:
from echotrain.evaluation.evaluator_ef import EFEvaluation
ef_ev = EFEvaluation(config) 
ef_ev.save_model(gbr,'gbr_test.sav')